In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2

import numpy as np
from collections import defaultdict


from PIL import Image, ImageFilter
import io
import re
import random
import numpy.random as npr
from skimage import data
from scipy.ndimage import rotate
from kernels import *
import torchvision
from torchvision import models
import os
from torchvision.transforms.functional import to_pil_image
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision.transforms as transforms
 
import my_utils as ut
import old_utils_averaged_filters as old_ut_avg
import old_utils_multi_image_old_snr as old_ut_multi
from transformers import Swinv2ForImageClassification, SwinConfig
from torch.optim import AdamW
from torchvision import transforms, datasets



In [2]:

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
   
])

# Create dataset instances
train_dataset = old_ut_multi.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='train')
val_test_dataset = old_ut_multi.DatasetAI(root_dir='/mnt/e/GenImage', transform=transform, split='val')


train_subset,val_subset, test_subset = ut.split_datasets(train_dataset, val_test_dataset, 30000, 3750, 3750)





train_loader = DataLoader(train_subset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False, num_workers=4)



In [3]:
#check distribution of classes 

print("Check overlap between datasets:\n")
print("\nTrain Subset and Validation Subset:")
ut.check_data_overlap(train_subset, val_subset)
print("\nTrain Subset and Test Subset:")
ut.check_data_overlap(train_subset, test_subset)
print("\nValidation Subset and Test Subset:")
ut.check_data_overlap(val_subset, test_subset)

print()
print("\nTrain Subset Distribution:\n")
ut.print_model_class_distribution(train_dataset, train_subset.indices)
print("\nValidation Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, val_subset.indices)
print("\nTest Subset Distribution:\n")
ut.print_model_class_distribution(val_test_dataset, test_subset.indices)

Check overlap between datasets:


Train Subset and Validation Subset:
No actual data overlap detected.

Train Subset and Test Subset:
No actual data overlap detected.

Validation Subset and Test Subset:
No actual data overlap detected.


Train Subset Distribution:

Total samples in subset: 296
Model ADM, Class ai: 20 (6.76%)
Model ADM, Class nature: 17 (5.74%)
Model BigGAN, Class nature: 20 (6.76%)
Model BigGAN, Class ai: 17 (5.74%)
Model Midjourney, Class ai: 21 (7.09%)
Model Midjourney, Class nature: 16 (5.41%)
Model VQDM, Class nature: 11 (3.72%)
Model VQDM, Class ai: 26 (8.78%)
Model glide, Class nature: 18 (6.08%)
Model glide, Class ai: 19 (6.42%)
Model stable_diffusion_v_1_4, Class nature: 22 (7.43%)
Model stable_diffusion_v_1_4, Class ai: 15 (5.07%)
Model stable_diffusion_v_1_5, Class nature: 19 (6.42%)
Model stable_diffusion_v_1_5, Class ai: 18 (6.08%)
Model wukong, Class ai: 22 (7.43%)
Model wukong, Class nature: 15 (5.07%)

Validation Subset Distribution:

Total samples in su

## old smash and reconstruct 

In [4]:
class BaseClassifier(nn.Module):
    def __init__(self,kernels):
        super(BaseClassifier, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        self.classifier = ut.DeepClassifier() 

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        outputs = self.classifier(feature_difference)

        return outputs
class ResNetClassifier(nn.Module):
    def __init__(self,kernels):
        super(ResNetClassifier, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        resnet_weights = models.ResNet50_Weights.DEFAULT
        self.resnet = models.resnet50(weights=resnet_weights)
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(self.resnet.children())[:-2])
        # Add a new classifier layer
        self.classifier = nn.Linear(self.resnet.fc.in_features, 2)
        # Adaptive pool to make sure output from feature maps is of fixed size
        self.adaptive_pool = nn.AdaptiveAvgPool2d((1, 1))

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        
        # Process feature difference through the ResNet features
        features = self.features(feature_difference)
        pooled_features = self.adaptive_pool(features)
        flat_features = torch.flatten(pooled_features, 1)
        outputs = self.classifier(flat_features)

        return outputs
class SwinClassification(nn.Module):
    def __init__(self,kernels):
        super(SwinClassification, self).__init__()
        self.feature_combiner = ut.CNNBlock(kernels)
        self.feature_combiner2 = ut.CNNBlock(kernels)
        config = SwinConfig.from_pretrained('microsoft/swinv2-tiny-patch4-window8-256',num_classes=2)
        self.transformer = Swinv2ForImageClassification.from_pretrained(
            "microsoft/swinv2-tiny-patch4-window8-256",
            config=config
        )
        
        self.transformer.classifier = nn.Linear(config.hidden_size, 2) 

 
    def forward(self, rich, poor):
       
        x = self.feature_combiner(rich)
        y = self.feature_combiner2(poor)   
        feature_difference = x - y
        outputs = self.transformer(feature_difference)

        return outputs.logits


# Frozen models old smash and reconstruct

In [5]:
#Swin Transformer
learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training SWIN Transformer Frozen w/OLDSNR\n")

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = SwinClassification(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_SWIN_frozen.pth'

    #freeze the transformer layers
    for param in model.transformer.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.transformer.classifier.parameters():
        param.requires_grad = True

    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.


Loaded previous best model with accuracy: 0.65625
Epoch 1/10
,Train Loss: 0.7018, Train Acc: 0.5338, Val Loss: 0.6918, Val Acc: 0.5625

Epoch 2/10
,Train Loss: 0.6967, Train Acc: 0.5203, Val Loss: 0.6534, Val Acc: 0.6562

Epoch 3/10
,Train Loss: 0.6697, Train Acc: 0.5912, Val Loss: 0.6615, Val Acc: 0.5625

Epoch 4/10
,Train Loss: 0.6707, Train Acc: 0.5574, Val Loss: 0.6542, Val Acc: 0.5312

Epoch 5/10
,Train Loss: 0.6710, Train Acc: 0.5980, Val Loss: 0.6550, Val Acc: 0.6562

Epoch 6/10
,Train Loss: 0.6563, Train Acc: 0.5946, Val Loss: 0.6484, Val Acc: 0.6875

Saved new best model with accuracy: 0.6875
Epoch 7/10
,Train Loss: 0.6446, Train Acc: 0.6284, Val Loss: 0.6524, Val Acc: 0.6875

Epoch 8/10
,Train Loss: 0.6461, Train Acc: 0.6351, Val Loss: 0.6413, Val Acc: 0.6250

Epoch 9/10
,Train Loss: 0.6407, Train Acc: 0.6081, Val Loss: 0.6477, Val Acc: 0.6875

Epoch 10/10
,Train Loss: 0.6393, Train Acc: 0.6216, Val Loss: 0.6384, Val Acc: 0.6562



In [6]:
#ResNET
learning_rates = [1e-3,1e-4, 1e-5, 1e-6]
print("Training RES Frozen w/OLDSNR\n")

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ResNetClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_RESNET_frozen.pth'
    
    #freeze resnet layers
    for param in model.resnet.parameters():
        param.requires_grad = False
    #unfreeze the classifier layer
    for param in model.classifier.parameters():
        param.requires_grad = True
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)
    
    

Loaded previous best model with accuracy: 0.65625
Epoch 1/10
,Train Loss: 0.7058, Train Acc: 0.4662, Val Loss: 0.6946, Val Acc: 0.4375

Epoch 2/10
,Train Loss: 0.6938, Train Acc: 0.4865, Val Loss: 0.6919, Val Acc: 0.5312

Epoch 3/10
,Train Loss: 0.6830, Train Acc: 0.5574, Val Loss: 0.6779, Val Acc: 0.6250

Epoch 4/10
,Train Loss: 0.6754, Train Acc: 0.6182, Val Loss: 0.6861, Val Acc: 0.5938

Epoch 5/10
,Train Loss: 0.6677, Train Acc: 0.6351, Val Loss: 0.6620, Val Acc: 0.5625

Epoch 6/10
,Train Loss: 0.6577, Train Acc: 0.6385, Val Loss: 0.6909, Val Acc: 0.5625

Epoch 7/10
,Train Loss: 0.6569, Train Acc: 0.6723, Val Loss: 0.6610, Val Acc: 0.5938

Epoch 8/10
,Train Loss: 0.6483, Train Acc: 0.6824, Val Loss: 0.6628, Val Acc: 0.5625

Epoch 9/10
,Train Loss: 0.6471, Train Acc: 0.6351, Val Loss: 0.6517, Val Acc: 0.6250

Epoch 10/10
,Train Loss: 0.6426, Train Acc: 0.6959, Val Loss: 0.6309, Val Acc: 0.6250



# unfreeze the pretrained models old reconstruct and destroy

In [7]:
#ResNET
print("Training RES UNFrozen w/OLDSNR\n")

learning_rates = [1e-3,1e-4, 1e-5, 1e-6]

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ResNetClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_RESNET_unfrozen.pth'


    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



Loaded previous best model with accuracy: 0.78125
Epoch 1/10
,Train Loss: 0.6910, Train Acc: 0.5439, Val Loss: 0.6638, Val Acc: 0.6562

Epoch 2/10
,Train Loss: 0.6215, Train Acc: 0.7095, Val Loss: 0.5954, Val Acc: 0.6875

Epoch 3/10
,Train Loss: 0.5688, Train Acc: 0.7196, Val Loss: 0.5796, Val Acc: 0.7500

Epoch 4/10
,Train Loss: 0.5056, Train Acc: 0.7635, Val Loss: 0.5723, Val Acc: 0.6562

Epoch 5/10
,Train Loss: 0.4423, Train Acc: 0.7939, Val Loss: 0.4840, Val Acc: 0.7188

Epoch 6/10
,Train Loss: 0.4088, Train Acc: 0.8243, Val Loss: 0.5002, Val Acc: 0.7812

Epoch 7/10
,Train Loss: 0.3268, Train Acc: 0.8986, Val Loss: 0.4145, Val Acc: 0.8438

Saved new best model with accuracy: 0.8438
Epoch 8/10
,Train Loss: 0.2740, Train Acc: 0.8750, Val Loss: 0.5531, Val Acc: 0.7812

Epoch 9/10
,Train Loss: 0.2369, Train Acc: 0.9122, Val Loss: 0.6411, Val Acc: 0.7500

Epoch 10/10
,Train Loss: 0.2105, Train Acc: 0.9358, Val Loss: 0.8387, Val Acc: 0.7188



In [8]:
#Swin Transformer
print("Training SWIN UNFrozen w/OLDSNR\n")

learning_rates = [1e-3,1e-4, 1e-5, 1e-6]

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    kernels =  ut.apply_high_pass_filter()
    model = SwinClassification(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_SWIN_Unfrozen.pth'
    
    
    
    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.


Loaded previous best model with accuracy: 0.78125
Epoch 1/10
,Train Loss: 0.7223, Train Acc: 0.5372, Val Loss: 0.6800, Val Acc: 0.5625

Epoch 2/10
,Train Loss: 0.6928, Train Acc: 0.5135, Val Loss: 0.6636, Val Acc: 0.5625

Epoch 3/10
,Train Loss: 0.6448, Train Acc: 0.5541, Val Loss: 0.6429, Val Acc: 0.7500

Epoch 4/10
,Train Loss: 0.6393, Train Acc: 0.6182, Val Loss: 0.6370, Val Acc: 0.5938

Epoch 5/10
,Train Loss: 0.6258, Train Acc: 0.6554, Val Loss: 0.6469, Val Acc: 0.5000

Epoch 6/10
,Train Loss: 0.6044, Train Acc: 0.6858, Val Loss: 0.6451, Val Acc: 0.6562

Epoch 7/10
,Train Loss: 0.6056, Train Acc: 0.6926, Val Loss: 0.7103, Val Acc: 0.5312

Epoch 8/10
,Train Loss: 0.6545, Train Acc: 0.5980, Val Loss: 0.6288, Val Acc: 0.6875

Epoch 9/10
,Train Loss: 0.5831, Train Acc: 0.6655, Val Loss: 0.6621, Val Acc: 0.6250

Epoch 10/10
,Train Loss: 0.5595, Train Acc: 0.6959, Val Loss: 0.5941, Val Acc: 0.6250



In [11]:
#Original Model
print("Training BASE w/OLDSNR\n")

learning_rates = [1e-3,1e-4, 1e-5, 1e-6]

for learning_rate in learning_rates:
    print(f"Training with Learning Rate: {learning_rate} \n\n")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BaseClassifier(kernels).to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    best_model_path = '/home/kosta/code/School/SentryAI/pth/best_model_oldSMR_Base.pth'


    ut.train_and_validate(model, train_loader, val_loader, optimizer, device, num_epochs=10, best_model_path= best_model_path)



No saved model found. Starting fresh!


Epoch 1/10
,Train Loss: 0.6858, Train Acc: 0.5811, Val Loss: 0.6958, Val Acc: 0.4688

Saved new best model with accuracy: 0.4688
Epoch 2/10
,Train Loss: 0.6708, Train Acc: 0.6216, Val Loss: 0.7015, Val Acc: 0.4688

Epoch 3/10
,Train Loss: 0.6657, Train Acc: 0.5980, Val Loss: 0.6885, Val Acc: 0.5938

Saved new best model with accuracy: 0.5938
Epoch 4/10
,Train Loss: 0.6448, Train Acc: 0.6284, Val Loss: 0.6584, Val Acc: 0.5938

Epoch 5/10
,Train Loss: 0.6305, Train Acc: 0.6318, Val Loss: 0.6088, Val Acc: 0.6562

Saved new best model with accuracy: 0.6562
Epoch 6/10
,Train Loss: 0.6191, Train Acc: 0.6520, Val Loss: 0.5881, Val Acc: 0.6562

Epoch 7/10
,Train Loss: 0.6249, Train Acc: 0.6284, Val Loss: 0.5646, Val Acc: 0.7500

Saved new best model with accuracy: 0.7500
Epoch 8/10
,Train Loss: 0.6116, Train Acc: 0.6486, Val Loss: 0.5392, Val Acc: 0.7812

Saved new best model with accuracy: 0.7812
Epoch 9/10
,Train Loss: 0.5975, Train Acc: 0.6858, Val Loss: 0.5595, Val Acc: 0.6562

Epoch 10/10